<a href="https://colab.research.google.com/github/Marteldelfer/PISI3/blob/main/regressao_receita_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports importantes

In [9]:
# --- CÉLULA 1: IMPORTAÇÃO DAS BIBLIOTECAS ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


Carregando Dataset

In [10]:
# --- CÉLULA 2: CARREGAMENTO DOS DADOS ---
try:
    dados = pd.read_csv('tmdb_new.csv')
    print("Arquivo 'tmdb_new.csv' carregado com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivo 'tmdb_new.csv' não encontrado. Faça o upload do arquivo.")

dados.head()

Arquivo 'tmdb_new.csv' carregado com sucesso.


,id,title,vote_average,vote_count,release_date,revenue,runtime,budget,original_language,popularity,...,production_countries,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,profit_percentage
0,5,Four Rooms,5.861,2687.0,1995-12-09,4257354.0,98.0,4000000.0,en,9.6284,...,United States of America,"Quinn Hellerman, Lawrence Bender, Tim Roth, La...","Allison Anders, Robert Rodriguez, Alexandre Ro...","Phil Parmet, Rodrigo García, Guillermo Navarro...","Allison Anders, Robert Rodriguez, Alexandre Ro...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,114082.0,6.433850
1,6,Judgment Night,6.500,349.0,1993-10-15,12136938.0,109.0,21000000.0,en,4.5607,...,United States of America,"Michael Scranton, Hank McGill, Donovan D. Ross...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Lloyd Segan, Marilyn Vance",Alan Silvestri,6.6,19885.0,-42.205057
2,11,Star Wars,8.204,21103.0,1977-05-25,775398007.0,121.0,11000000.0,en,67.2931,...,United States of America,"Paul Blake, Hal Wamsley, David Prowse, Larry W...",George Lucas,Gilbert Taylor,George Lucas,"Gary Kurtz, George Lucas, Rick McCallum",John Williams,8.6,1504993.0,6949.072791
3,12,Finding Nemo,7.815,19616.0,2003-05-30,940335536.0,100.0,94000000.0,en,18.2106,...,United States of America,"Jeff Pidgeon, Danny Mann, Jessie Flower, Bill ...",Andrew Stanton,"Sharon Calahan, Jeremy Lasky","Bob Peterson, Will Csaklos, Ronnie del Carmen,...","John Lasseter, Graham Walters",Thomas Newman,8.2,1158248.0,900.356953
4,13,Forrest Gump,8.468,28113.0,1994-06-23,677387716.0,142.0,55000000.0,en,27.4853,...,United States of America,"Margo Moorer, Joe Stefanelli, Ed Davis, Elizab...",Robert Zemeckis,Don Burgess,"Winston Groom, Eric Roth","Wendy Finerman, Steve Tisch, Steve Starkey",Alan Silvestri,8.8,2374308.0,1131.614029


In [11]:
# --- CÉLULA 3: LIMPEZA DE DADOS E RESET DE ÍNDICE ---
print(f"Formato original: {dados.shape}")

# Remove colunas que não serão usadas
colunas_inuteis = ['Unnamed: 0', 'id']
for col in colunas_inuteis:
    if col in dados.columns:
        dados = dados.drop(col, axis=1)
        print(f"Coluna '{col}' removida.")

# Remove linhas com dados faltantes
dados.dropna(inplace=True)
print("Linhas com dados nulos removidas.")

dados.reset_index(drop=True, inplace=True)

print(f"Formato após limpeza e reset: {dados.shape}")

Formato original: (12020, 22)
Coluna 'id' removida.
Linhas com dados nulos removidas.
Formato após limpeza e reset: (8435, 21)


In [12]:
# --- CÉLULA 4: CLUSTERING PARA CRIAÇÃO DE FEATURES ---
print("Iniciando o processo de clustering...")

features_cluster = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']
dados_para_cluster = dados[features_cluster]

scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dados_para_cluster)
print("Dados normalizados.")

kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
clusters = kmeans.fit_predict(dados_scaled)
print("Modelo KMeans treinado.")

dados['cluster'] = clusters
print("Coluna 'cluster' adicionada com sucesso ao DataFrame.")

Iniciando o processo de clustering...
Dados normalizados.
Modelo KMeans treinado.
Coluna 'cluster' adicionada com sucesso ao DataFrame.


Criar e salvar matrizes esparsas

In [13]:
# --- CÉLULA 5: PREPARAÇÃO FINAL PARA O MODELO---

print("Preparando os dados para o modelo de regressão...")

# 1. Transforma a coluna 'cluster' em colunas numéricas (0 ou 1)
dados = pd.get_dummies(dados, columns=['cluster'], prefix='cluster')
print("Coluna 'cluster' transformada em features numéricas.")

# 2. Define y (o alvo)
y = dados['revenue']

# 3. Define X (features)
features_numericas = dados.select_dtypes(include=np.number).columns.tolist()

# A lista de features numéricas inclui nosso alvo 'revenue'.
if 'revenue' in features_numericas:
    features_numericas.remove('revenue')

X = dados[features_numericas]

print("\nVariáveis X (features) e y (alvo) definidas de forma segura.")
print(f"X agora contém apenas as {X.shape[1]} colunas numéricas.")

Preparando os dados para o modelo de regressão...
Coluna 'cluster' transformada em features numéricas.

Variáveis X (features) e y (alvo) definidas de forma segura.
X agora contém apenas as 8 colunas numéricas.


In [14]:
# --- CÉLULA 6: DIVISÃO EM DADOS DE TREINO E TESTE ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dados divididos em conjuntos de treino e teste.")

Dados divididos em conjuntos de treino e teste.


In [15]:
# --- CÉLULA 7: TREINAMENTO DO MODELO DE REGRESSÃO ---
modelo = RandomForestRegressor(random_state=42, n_estimators=100)
print("Iniciando o treinamento do modelo RandomForestRegressor...")

modelo.fit(X_train, y_train)

print("Modelo treinado com sucesso!")

Iniciando o treinamento do modelo RandomForestRegressor...
Modelo treinado com sucesso!


In [16]:
# --- CÉLULA 8: AVALIAÇÃO DA PERFORMANCE DO MODELO ---
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- Resultados da Avaliação do Modelo ---")
print(f"Erro Médio Absoluto (MAE): ${mae:,.2f}")
print(f"Raiz do Erro Quadrático Médio (RMSE): ${rmse:,.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2f}")


--- Resultados da Avaliação do Modelo ---
Erro Médio Absoluto (MAE): $5,492,825.48
Raiz do Erro Quadrático Médio (RMSE): $37,984,505.57
Coeficiente de Determinação (R²): 0.95
